In [1]:
import cv2 as cv
import numpy as np

In [2]:
cap = cv.VideoCapture('../src/barcodes.avi')
cv.namedWindow('Códigos de Barra Detectados', cv.WINDOW_NORMAL)

while True:
    ret, frame = cap.read()
    
    if not ret:
        break

    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    gradX = cv.Sobel(gray, ddepth=cv.CV_32F, dx=1, dy=0, ksize=-1)
    gradY = cv.Sobel(gray, ddepth=cv.CV_32F, dx=0, dy=1, ksize=-1)

    gradient = cv.subtract(gradX, gradY)
    gradient = cv.convertScaleAbs(gradient)

    blurred = cv.blur(gradient, (9, 9))
    (_, thresh) = cv.threshold(blurred, 225, 255, cv.THRESH_BINARY)

    # Operações morfológicas para fechar buracos
    kernel = cv.getStructuringElement(cv.MORPH_RECT, (21, 7))
    closed = cv.morphologyEx(thresh, cv.MORPH_CLOSE, kernel)

    closed = cv.erode(closed, None, iterations=4)
    closed = cv.dilate(closed, None, iterations=4)

    cnts, _ = cv.findContours(closed.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    if cnts:
        # Pegar o maior contorno
        c = sorted(cnts, key=cv.contourArea, reverse=True)[0]

        x, y, w, h = cv.boundingRect(c)

        if w > h:  # Horizontal
            cv.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), -1)  # Azul para horizontal
        else:  # Vertical
            cv.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), -1)  # Vermelho para vertical

    cv.imshow('Códigos de Barra Detectados', frame)

    if cv.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/kaio/.local/lib/python3.10/site-packages/cv2/qt/plugins"
